In [1]:
import os
from IPython.display import FileLink
import networkx as nx
import csv
import math
from collections import namedtuple
from sklearn.cluster import KMeans
import random
import heapq
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
# define la longitud del recorrido, incluye el regreso al punto de partida

def tourLength(points, solution, nodeCount):
    obj = length(points[solution[-1]], points[solution[0]])
    for index in range(0, nodeCount-1):
        obj += length(points[solution[index]], points[solution[index+1]])

    return obj


In [4]:
#Comprueba que se visiten todas las localizacion, y que ninguna se repita

def check_solution(node_count, solution, points):
    solutionCheck = sorted(solution, key=lambda x: x)

    for i in range(node_count):
        if solutionCheck[i] != i:
            print("solución inválida, no se recorren todas la ciudades")
            return 0

    return tourLength(points, solution, node_count)

In [5]:
#Ordena los puntos segun proxmidad al centro de masa de la nube de puntos

def edgesFromCentroid(orden):
    x = 0
    y = 0

    for point in orden:
        x += point[0]
        y += point[1]

    meanPoint = Point(x/len(orden), y / len(orden))
    orden = sorted(orden, key=lambda p: -length(p, meanPoint))
    return orden


In [6]:
#Ordena los puntos segun la proximidad a un punto fuera de la nube de puntos

def edgesFromOutlayer(orden):
    x = 0
    y = 0

    for point in orden:
        x += point[0]
        y += point[1]

    Outlayer = Point(x, y)
    orden = sorted(orden, key=lambda p: -length(p, Outlayer))
    return orden

In [7]:
# Centro de masas de la nube de puntos

def centroide(points):
    x = []
    for i in range(len(points)):
        x.append([points[i].x, points[i].y])

    kmeans = KMeans(n_clusters=1).fit(x)
    centroids = kmeans.cluster_centers_
    return centroids[0]


In [8]:
def vectorDirector(a, b):
    return Point(b[0] - a[0], b[1] - a[0])

In [9]:
def anguloRectas(u, v):
    cosenoAngulo = (u[0] * v[0]) + (u[1] * v[1]) / (math.sqrt(u[0]**2 + u[1]**2) * (math.sqrt(v[0]**2 + v[1]**2)) + 0.0000000000000000001)
    return math.acos(cosenoAngulo)

In [10]:
# Pasos:
# Arbol expansion minimo
# coger nodos de grado impar
# realizar emparejamiento perfecto entre los de grado impar
# añadir al arbol las aristas del emparejamiento
# recorrer los nodos en preorden


def christofides(nodeCount, G):
    
    T = nx.MultiGraph()
    minumumTree = nx.minimum_spanning_tree(G).edges

    T.add_edges_from(minumumTree)
    W = nx.Graph()

    for i in range(nodeCount):
        if T.degree[i] % 2 != 0:
            W.add_edges_from(T.edges(i))

    if len(W) > 0:
        M = dict(nx.all_pairs_shortest_path_length(W))
        if '1' in M:
            T.add_edges_from(M[1].items())

    return list(nx.dfs_preorder_nodes(T))

In [11]:
# Crea conexiones entre los nodos segun como esten ordenados
# cuanto mas nos acercamos al final menos conexiones tienen
# la idea es crear un grafo no completo, lo suficientemente conectado
# para que el arbol de expansion minimo sea el optimo

def incomplexGraf(nodeCount, points):
    G = nx.MultiGraph()

    for i in range(nodeCount - 1):
        for j in range(i + 1, i + 100):
            if j < nodeCount:
                G.add_edge(i, j, weight=length(points[i], points[j]))
    return G

In [12]:
# Se realiza el circuito, y a cada paso se intenta invertir el orden
# del proximo ciclo de dos nodos, buscando minimizar el recorrido total 
# del mismo, la idea es no tener aristas que se cruzen


def twoOpt(solution, points):
    longA = tourLength(points, solution, len(solution))
    
    for i in range(len(solution) - 2):
        aux = solution[i + 1]
        solution[i + 1] = solution[i + 2]
        solution[i + 2] = aux
        longB = tourLength(points, solution, len(solution))

        if longA > longB:
            longA = longB
        else:
            aux = solution[i + 1]
            solution[i + 1] = solution[i + 2]
            solution[i + 2] = aux

    return solution

In [13]:
# coste Simulating Anniling

def costSA(theta):
    return -(theta/math.log(0.9))

In [14]:
# A la hora de crear soluciones vecinas, algunas ciudades pueden
# quedar juntas por un minimo local, de esta manera, se intenta evitar,
# al intercambiar la posicion de dos ciudades al "azar"

def mutationSolution(solution):
    a = random.randint(0, len(solution) - 1)
    b = random.randint(0, len(solution) - 1)
    aux = solution[a]
    solution[a] = solution[b]
    solution[b] = aux

In [15]:
# Generacion de una solucion vecina a partir de dos soluciones
# Elegimos un punto de corte al azar
# rellenamos con el recorrido A
# A partir del punto de corte rellenamos con la solucion B

def legit(solution, Msolution, nodeCount):
    ini = random.randint(0, nodeCount - 1)
    j = 0
    hlegit = []

    for i in range(ini):
        hlegit.append(solution[i])

    while nodeCount > len(hlegit):
        if Msolution[j] not in hlegit:
            hlegit.append(Msolution[j])
        j = (j + 1) % nodeCount    # convercion a "array circular"

    return hlegit

In [16]:
# Generacion de solucion vecina, usando el valor guardado en A,
# como indice para B, tiene el problema de que puede crear un ciclo
# de la solucion entera, haciendo que todos los decendientes sean simetricos

def indexGen(solution, Msolution):
    ini = random.randint(0, len(solution) - 1)
    hlegit = solution.copy()

    for i in range(len(solution)):
        hlegit[ini] = Msolution[solution[i]]
        ini = (ini + 1) % len(solution)

    return hlegit

In [17]:
# Calcula cuanto aporta cada solucion al total
# Los que mas aportan tienen una mayor posibilidad de
# sobrevivir, aun asi a los super individuos que han llegado a
# minimos locales pueden no sobrevivir, ya que aleatoriamente
# se les resta a su probabilidad un numero aleatorio


def bestAdaptation(adaptation):
    accuracy = adaptation.values()
    total = sum(accuracy)

    probabilidad = []

    for i in accuracy:
        probabilidad.append((1 - (i / total)) - random.uniform(0, 1)) # probabilidad de sobrevivir

    newGen = heapq.nlargest(2, range(len(probabilidad)), key=probabilidad.__getitem__)
    adaptation["newpadre"] = newGen[0] # mejores adaptados
    adaptation["newmadre"] = newGen[1]

In [18]:
def TSPGenetico(points, solution):
    
    nodeCount = len(solution)
    adaptation = {}
    padre = solution
    value = check_solution(nodeCount, padre, points)
    solutions = padre[0: 4] # numero de hijos por generacion


    madre = padre.copy() # random de otra solucion
    random.shuffle(madre)

    solutions[0] = padre
    solutions[1] = madre

    
    for i in range(100):   # numero de generaciones
        adaptation.clear()

        adaptation["padre"] = check_solution(nodeCount, solutions[0], points)
        adaptation["madre"] = check_solution(nodeCount, solutions[1], points)

        for i in range(2, 4 - 2, 2):  # numero de hijos

            solutions[i] = indexGen(solutions[0], solutions[1])
            adaptation["bastard"] = check_solution(nodeCount, solutions[i], points)

            solutions[i + 1] = legit(solutions[0], solutions[1], nodeCount)
            adaptation["legit"] = check_solution(nodeCount, solutions[i + 1], points)


        bestAdaptation(adaptation)       # evaluacion de la generacion
        solutions[0] = solutions[adaptation["newpadre"]]
        solutions[1] = solutions[adaptation["newmadre"]]

        if adaptation["padre"] < value: # guardamos al mejor de todas la generaciones
            value = adaptation["padre"]
            solution = solutions[0].copy()

        if random.uniform(0, 1) < 0.001: # indice de mutacion
            mutationSolution(solutions[0])
            mutationSolution(solutions[1])

    return solution

In [19]:
# la idea es hacer un barrido circular desde el cento de masa de la nube de puntos, para calcular un recorrido
# su funcionamiento depende tanto de la densidad de la nube, como el buen posicionamiento de centro, a mayor 
# densidad mas se aleja de una solucion satisfactoria, se nos ha ocurrido una idea para solventarlo, que consistiria en 
# varios barridos y conectar los nodos que esten a cierta distancia del centro, haciendo una especie de anillos concentricos,
# aunque por tiempo no hemos podido

def TSPunionAnguloCentroMasa(points, solution):    
    
    cent = centroide(points)
    meanPoint = Point(cent[0], cent[1])
    meanVector = vectorDirector(meanPoint, Point(cent[0], cent[1] + 5)) # recta vertical que pasa por el cento de masa
    
    
     # se ordena la solucion segun el angulo que forma la recta vertical, con la recta que pasa por el centro de masa y el punto
    solution = sorted(solution, key=lambda x: anguloRectas(meanVector, vectorDirector(meanPoint, points[x])))


    return solution

In [21]:
def TSPchristofides(points, solution):
    nodeCount = len(solution)

    if nodeCount < 2000:
        G = nx.MultiGraph()
        # Para grafos de menos de 2000 nodos, creamos el grafo completo
        
        for i in range(nodeCount):
            for j in range(i + 1, nodeCount):
                G.add_edge(i, j, weight=length(points[i], points[j]))

        solution = christofides(nodeCount, G)
        return solution

    
    # segun estan ordenados, dato base sobre el que se intenta mejorar

    G = incomplexGraf(nodeCount, points)
    solution = christofides(nodeCount, G)
    value = check_solution(nodeCount, solution, points)
    
    #Ordenacion fuera nube puntos

    G = incomplexGraf(nodeCount, edgesFromOutlayer(points))
    solutionout = christofides(nodeCount, G)
    valueout = check_solution(nodeCount, solutionout, points)
    
    #Ordenacion centro masa

    G = incomplexGraf(nodeCount, edgesFromCentroid(points))
    solutionCent = christofides(nodeCount, G)
    valueCent = check_solution(nodeCount, solutionCent, points)
    
    
        # comprobacion de la mejor estrategia
        
    if value > valueout:
        value = valueout
        solution = solutionout

    if value > valueCent:
        value = valueCent
        solution = solutionCent
        
    return solution

In [22]:
def submission_generation(filename, str_output):
    os.chdir(r'/kaggle/working')
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        for item in str_output:
            writer.writerow(item)
    return  FileLink(filename)

In [23]:
Point = namedtuple("Point", ['x', 'y'])

 # longitud entre dos puntos
    
def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

In [24]:
def solve_it(input_data):

    lines = input_data.split('\n')

    nodeCount = int(lines[0])

    solution = []
    points = []

    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
        solution.append(i-1)

        
    solution = TSPchristofides(points, solution)
    
    #solution = TSPsimulatingAnneling(points, solution, alpha = 0.9)
    
    #solution = TSPunionAnguloCentroMasa(points, solution)
    
    #solution = TSPGenetico(points, solution)
    
    value = check_solution(nodeCount, solution, points)
        
        

    # prepare the solution in the specified output format
    output_data = '%.2f' % value + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, solution))

    return output_data, value

In [25]:
class City():
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    
    def distance(self, city):
        xdist = abs(self.x - city.x)
        ydist = abs(self.y - city.y)
        res = np.sqrt((xdist ** 2) + (ydist ** 2))
        return res


    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [26]:
def parse_input(raw_data):
    # parse the input
    lines = input_data.split('\n')

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(City(float(parts[0]), float(parts[1])))
    
    return nodeCount, points

In [27]:
import matplotlib.pyplot as plt

class SimAnneal():
    def __init__(self, cities, T=-1, alpha=-1, stopping_T=-1, stopping_iter=-1):
        self.cities = cities
        self.N = len(cities)
        self.T = np.sqrt(self.N) if T == -1 else T
        self.T_save = self.T  # save inital T to reset if batch annealing is used
        self.alpha = 0.995 if alpha == -1 else alpha
        self.stopping_temperature = 1e-8 if stopping_T == -1 else stopping_T
        self.stopping_iter = 100000 if stopping_iter == -1 else stopping_iter
        self.iteration = 1

        self.nodes = [i for i in range(self.N)]

        self.best_solution = None
        self.best_fitness = float("Inf")
        self.fitness_list = []
    

    def initial_solution(self):
        # Greedy algorithm to get an initial solution (closest-neighbour).
        cur_node = np.random.choice(self.nodes)  # start from a random node
        solution = [cur_node]

        free_nodes = set(self.nodes)
        free_nodes.remove(cur_node)
        while free_nodes:
            next_node = min(free_nodes, key=lambda x: self.dist(cur_node, x))  # nearest neighbour
            free_nodes.remove(next_node)
            solution.append(next_node)
            cur_node = next_node

        cur_fit = self.fitness(solution)
        if cur_fit < self.best_fitness:  # If best found so far, update best fitness
            self.best_fitness = cur_fit
            self.best_solution = solution
        self.fitness_list.append(cur_fit)
        return solution, cur_fit


    def dist(self, node_0, node_1):
        return self.cities[node_0].distance(self.cities[node_1])


    def fitness(self, solution):
        # Total distance of the current solution path.
        cur_fit = 0
        for i in range(self.N):
            cur_fit += self.dist(solution[i % self.N], solution[(i + 1) % self.N])
        return cur_fit


    def p_accept(self, candidate_fitness):
        # Probability of accepting if the candidate is worse than current.
        # Depends on the current temperature and difference between candidate and current.
        return np.exp(-abs(candidate_fitness - self.cur_fitness) / self.T)


    def accept(self, candidate):
        # Accept with probability 1 if candidate is better than current.
        # Accept with probabilty p_accept(..) if candidate is worse.
        candidate_fitness = self.fitness(candidate)
        if candidate_fitness < self.cur_fitness:
            self.cur_fitness, self.cur_solution = candidate_fitness, candidate
            if candidate_fitness < self.best_fitness:
                self.best_fitness, self.best_solution = candidate_fitness, candidate
        
        elif np.random.random() < self.p_accept(candidate_fitness):
                self.cur_fitness, self.cur_solution = candidate_fitness, candidate


    def anneal(self):
        # Execute simulated annealing algorithm.
        # Initialize with the greedy solution.
        self.cur_solution, self.cur_fitness = self.initial_solution()

        # print("Starting annealing.")
        while self.T >= self.stopping_temperature and self.iteration < self.stopping_iter:
            candidate = list(self.cur_solution)
            l = np.random.randint(2, self.N - 1)
            i = np.random.randint(0, self.N - l)
            candidate[i : (i + l)] = reversed(candidate[i : (i + l)])
            self.accept(candidate)
            self.T *= self.alpha
            self.iteration += 1

            self.fitness_list.append(self.cur_fitness)
        return self.best_solution


    def plot_learning(self):
        # Plot the fitness through iterations.
        plt.plot(range(len(self.fitness_list)), self.fitness_list)
        plt.ylabel("Fitness")
        plt.xlabel("Iteration")
        plt.show()

In [28]:
def annealing_sol(nodeCount, cities, plot=False):
    # build a trivial solution
    # visit the nodes in the order they appear in the file
    # solution = range(0, nodeCount)
    sa = SimAnneal(cities, stopping_T=1e-10)
    
    sa.anneal()
    if plot:
        sa.plot_learning()

    return sa.best_solution

In [29]:
def solve_it_annealing(input_data, plot=False):
    nodeCount, cities = parse_input(input_data)

    solution = annealing_sol(nodeCount, cities, plot)


    # calculate the length of the tour
    obj = cities[solution[-1]].distance(cities[solution[0]])
    for index in range(0, nodeCount-1):
        obj += cities[solution[index]].distance(cities[solution[index+1]])

    # prepare the solution in the specified output format
    output_data = '%.2f' % obj + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, solution))

    return output_data, obj

In [30]:
str_output = [["Filename","Value"]]
valueT = 0
for dirname, _, filenames in os.walk('/kaggle/input/tsp-aco'):
    for filename in filenames:
        full_name = dirname+'/'+filename
        with open(full_name, 'r') as input_data_file:
            input_data = input_data_file.read()
            output, value = solve_it_annealing(input_data)
            print(value)
            valueT += value
            str_output.append([filename,str(value)])
            print("11111111111111111111111111111")

print(valueT)

25248.0101361898
11111111111111111111111111111
61107.1094872115
11111111111111111111111111111
100285.76381761527
11111111111111111111111111111
114238.48473555515
11111111111111111111111111111
60598.92700047736
11111111111111111111111111111
2681.895570679105
11111111111111111111111111111
4.0
11111111111111111111111111111
422089.0348235135
11111111111111111111111111111
64883.3434555518
11111111111111111111111111111
11398.310320540615
11111111111111111111111111111
77724.57614499841
11111111111111111111111111111
42407.66676663639
11111111111111111111111111111
1126128.4814429258
11111111111111111111111111111
59105.96673916231
11111111111111111111111111111
31817.39777447803
11111111111111111111111111111
225730.13438877137
11111111111111111111111111111
53013.8469012421
11111111111111111111111111111
45747.695895094344
11111111111111111111111111111
79618.88487768389
11111111111111111111111111111
471472.57154951466
11111111111111111111111111111
26897.572058320562
11111111111111111111111111111
33

In [31]:
submission_generation('simulated_annealing.csv', str_output)

/kaggle/working/simulated_annealing.csv

In [32]:
reader = csv.reader(open("sample_submission_non_sorted.csv"))
sortedlist = sorted(reader, key=lambda row: row[0], reverse=False)

In [33]:
submission_generation('sample_submission.csv', sortedlist)

In [34]:
"""
include "globals.mzn";
int : node_count;

set of int: nodes = 1..node_count;
array [nodes] of var nodes: orden;
array [nodes, 1..2] of float: map;


constraint forall(i,j in nodes where j > i)(orden[i] != orden[j]);
constraint forall(i,j in nodes where j > i)(orden[j] != orden[i]);
constraint alldifferent([orden[i] | i in nodes]);
%constraint circuit([orden[i] | i in nodes]);

%constraint forall(i,k in 1..node_count where k > i)(orden[i] = i + arg_min([abs(map[orden[i],1] - map[orden[j],1]) + abs(map[orden[i],2] - map[orden[j],2]) | j in k..node_count]));


constraint orden[1] = 1;

var float: regreso = abs(map[orden[node_count],1] - map[orden[1],1]) + abs(map[orden[node_count],2] - map[orden[1],2]);

var float: recorrido = sum(i in 1..node_count-1)(abs(map[orden[i],1] - map[orden[i+1],1]) + abs(map[orden[i],2] - map[orden[i+1],2])) + regreso;

%solve minimize recorrido;
%solve satisfy;
solve :: int_search(orden, first_fail, indomain_min, complete) minimize recorrido;

"""

'\ninclude "globals.mzn";\nint : node_count;\n\nset of int: nodes = 1..node_count;\narray [nodes] of var nodes: orden;\narray [nodes, 1..2] of float: map;\n\n\nconstraint forall(i,j in nodes where j > i)(orden[i] != orden[j]);\nconstraint forall(i,j in nodes where j > i)(orden[j] != orden[i]);\nconstraint alldifferent([orden[i] | i in nodes]);\n%constraint circuit([orden[i] | i in nodes]);\n\n%constraint forall(i,k in 1..node_count where k > i)(orden[i] = i + arg_min([abs(map[orden[i],1] - map[orden[j],1]) + abs(map[orden[i],2] - map[orden[j],2]) | j in k..node_count]));\n\n\nconstraint orden[1] = 1;\n\nvar float: regreso = abs(map[orden[node_count],1] - map[orden[1],1]) + abs(map[orden[node_count],2] - map[orden[1],2]);\n\nvar float: recorrido = sum(i in 1..node_count-1)(abs(map[orden[i],1] - map[orden[i+1],1]) + abs(map[orden[i],2] - map[orden[i+1],2])) + regreso;\n\n%solve minimize recorrido;\n%solve satisfy;\nsolve :: int_search(orden, first_fail, indomain_min, complete) minimize r